# Домашнее задание к уроку 2. Создание признакового пространства

все материалы для выполения дз в `sem2.ipynb`

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('/content/drive/MyDrive/positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('/content/drive/MyDrive/negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [9]:
df.tail()

,text,label
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative
111922,Такси везет меня на работу. Раздумываю приплат...,negative


In [10]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [11]:
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
from nltk import ngrams
import nltk
from nltk import collocations 
nltk.download('genesis')
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Unzipping corpora/genesis.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
from string import punctuation
noise = stopwords.words('russian') + list(punctuation)

In [16]:
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
corpus = [token for tweet in df.text for token in word_tokenize(tweet) if token not in punctuation]
print(len(corpus))
corpus[:10]

2870536


['first_timee', 'хоть', 'я', 'и', 'школота', 'но', 'поверь', 'у', 'нас', 'то']

In [18]:
freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:10]

[('не', 69472),
 ('и', 55166),
 ('в', 52902),
 ('я', 52818),
 ('RT', 38070),
 ('на', 35759),
 ('http', 32998),
 ('что', 31541),
 ('с', 27217),
 ('а', 26860)]

## Задание 1.

**Задание**: обучите три классификатора: 

1) на токенах с высокой частотой 

2) на токенах со средней частотой 

3) на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

In [19]:
# Подготовим три группы токенов: с высокой, средней и низкой частотой

high_tokens = set()
med_tokens = set()
lit_tokens = set()
h = 10000 # если частота выше этого значения будем считать что это токен с высокой частотой
m = 500 # если частота ниже этого значения будем считать что это токен с низкой частотой
for i in freq_dict_sorted:
    if i[1] > h:
        high_tokens.add(i[0])
    elif i[1] < m:
        lit_tokens.add(i[0])
    else:
        med_tokens.add(i[0])
print(len(high_tokens),len(med_tokens),len(lit_tokens))

29 507 350587


In [20]:
# создадим фильтр (стопслова) с добавлением токенов средней и низкой частоты, 
# то есть будем считать только по токенам с высокой частотой
sw = noise + list(lit_tokens) + list(med_tokens)

In [21]:
import warnings
warnings.filterwarnings("ignore")

In [22]:
%%time
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=sw)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.80      0.57      0.67     38964
    positive       0.42      0.69      0.53     17745

    accuracy                           0.61     56709
   macro avg       0.61      0.63      0.60     56709
weighted avg       0.68      0.61      0.62     56709

CPU times: user 1min 20s, sys: 6.3 s, total: 1min 27s
Wall time: 1min 22s


In [24]:
# посчитаем только по токенам с средней частотой
sw = noise + list(high_tokens) + list(lit_tokens)

In [25]:
%%time
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=sw)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.57      0.68      0.62     23194
    positive       0.74      0.64      0.69     33515

    accuracy                           0.66     56709
   macro avg       0.65      0.66      0.65     56709
weighted avg       0.67      0.66      0.66     56709

CPU times: user 1min 20s, sys: 5.73 s, total: 1min 26s
Wall time: 1min 21s


In [26]:
# посчитаем только по токенам с низкой частотой
sw = noise + list(high_tokens) + list(med_tokens)

In [27]:
%%time
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=sw)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.79      0.72      0.75     30422
    positive       0.71      0.78      0.74     26287

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.75     56709
weighted avg       0.75      0.75      0.75     56709

CPU times: user 59.8 s, sys: 10.9 s, total: 1min 10s
Wall time: 1min


**Вывод:** Лучшая точность получилась на токенах с низкой частотностью, имеется зависимость от смещения границ разделения частотности, если за границу брать количественное значение частотности, лучший результат дает низкая частотность.

## О важности эксплоративного анализа

Но иногда пунктуация бывает и не шумом -- главное отталкиваться от задачи. Что будет если вообще не убирать пунктуацию?

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27623
    positive       1.00      1.00      1.00     29086

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Шок! Стоило оставить пунктуацию -- и все метрики равны 1. Как это получилось? Среди неё были очень значимые токены (как вы думаете, какие?). Найдите фичи с самыми большими коэффициэнтами:

## Задание 2.

найти фичи с наибольшей значимостью, и вывести их

In [29]:
# from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
# Оставим знаки пунктуации  
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=stopwords.words('russian'))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27628
    positive       1.00      1.00      1.00     29081

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



In [31]:
# существенное влияние оказывают симолы, участвующие в смайлах:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [32]:
p = list('"#$%&\'+,.<=>?@[\\]^`{|}~')
p

['"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '+',
 ',',
 '.',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '`',
 '{',
 '|',
 '}',
 '~']

In [33]:
myfilter = stopwords.words('russian') + p
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words= myfilter)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27614
    positive       1.00      1.00      1.00     29095

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Посмотрим, как один из супер-значительных токенов справится с классификацией безо всякого машинного обучения:

In [34]:
cool_token = ')'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92     32683
    positive       0.83      1.00      0.91     24026

    accuracy                           0.91     56709
   macro avg       0.91      0.92      0.91     56709
weighted avg       0.93      0.91      0.91     56709



In [35]:
pp = []
for i in punctuation:
    cool_token = i
    pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
    pp.append((i,accuracy_score(pred, y_test)))
print(pp)

[('!', 0.5115061101412475), ('"', 0.4998677458604454), ('#', 0.4975224391190111), ('$', 0.48835281877656106), ('%', 0.49048651889470807), ('&', 0.48770036502142516), ("'", 0.48819411380909555), ('(', 0.025286991482833414), (')', 0.9118658414008358), ('*', 0.5087728579237863), ('+', 0.48870549648203987), (',', 0.5024246592251671), ('-', 0.5038353700470825), ('.', 0.5074150487576928), ('/', 0.5387857306600363), (':', 0.5439524590453014), (';', 0.49247914793066355), ('<', 0.48819411380909555), ('=', 0.4883880865471089), ('>', 0.48819411380909555), ('?', 0.5005731012714031), ('@', 0.5675642314271103), ('[', 0.48822938157964346), ('\\', 0.4882117476943695), (']', 0.4883351848912871), ('^', 0.4946833835899064), ('_', 0.514997619425488), ('`', 0.48794723941526036), ('{', 0.4881764799238216), ('|', 0.4842970251635543), ('}', 0.4881764799238216), ('~', 0.4880001410710822)]


Выведем фичи с наибольшей значимостью

In [36]:
for i in sorted(pp, key=lambda x: -x[1])[:10]:
    print(i[0],'  -   ' ,i[1])

)   -    0.9118658414008358
@   -    0.5675642314271103
:   -    0.5439524590453014
/   -    0.5387857306600363
_   -    0.514997619425488
!   -    0.5115061101412475
*   -    0.5087728579237863
.   -    0.5074150487576928
-   -    0.5038353700470825
,   -    0.5024246592251671


## Символьные n-граммы

Теперь в качестве фичей используем, например, униграммы символов:

In [37]:
vec = CountVectorizer(analyzer='char', ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.99      1.00      1.00     27589
    positive       1.00      0.99      1.00     29120

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



В общем-то, теперь уже понятно, почему на этих данных здесь 1. Так или инчае, на символах классифицировать тоже можно: для некторых задач (например, для определения языка) фичи-символьные n-граммы решительно рулят.

Ещё одна замечательная особенность фичей-символов: токенизация и лемматизация не нужна, можно использовать такой подход для языков, у которых нет готвых анализаторов.

## Задание 3.

1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера 

3) убедиться что для сетки нет переобучения

In [43]:
%%time
myfilter = stopwords.words('russian') + p

count_vect = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words= myfilter)
bow = count_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

ValueError: ignored

In [44]:
%%time
myfilter = stopwords.words('russian')
tf_vect = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words= myfilter)
bow = tf_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(tf_vect.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27628
    positive       1.00      1.00      1.00     29081

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709

CPU times: user 58.1 s, sys: 7.65 s, total: 1min 5s
Wall time: 58.4 s


In [45]:
from sklearn.feature_extraction.text import HashingVectorizer

In [46]:
vectorizer = HashingVectorizer(n_features=2**4,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.52      0.53      0.53     26793
    positive       0.57      0.55      0.56     29916

    accuracy                           0.54     56709
   macro avg       0.54      0.54      0.54     56709
weighted avg       0.55      0.54      0.54     56709



In [47]:
vectorizer = HashingVectorizer(n_features=2**8,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.60      0.60      0.60     27628
    positive       0.62      0.62      0.62     29081

    accuracy                           0.61     56709
   macro avg       0.61      0.61      0.61     56709
weighted avg       0.61      0.61      0.61     56709



In [48]:
vectorizer = HashingVectorizer(n_features=2**10,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.64      0.65      0.64     27276
    positive       0.67      0.66      0.66     29433

    accuracy                           0.65     56709
   macro avg       0.65      0.65      0.65     56709
weighted avg       0.65      0.65      0.65     56709



In [49]:
vectorizer = HashingVectorizer(n_features=2**12,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.67      0.69      0.68     27141
    positive       0.71      0.70      0.70     29568

    accuracy                           0.69     56709
   macro avg       0.69      0.69      0.69     56709
weighted avg       0.69      0.69      0.69     56709

